In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import networkx as nx
import numpy as np
import pandas as pd

# Load the CSV file into a DataFrame
nodes_df = pd.read_csv('nodes.csv')
# Load the edges.csv file into a DataFrame
edges_df = pd.read_csv('edges_flows.csv')

def create_graph(nodes_df, edges_df, migration_year):
    graph = nx.DiGraph()

    # Add nodes
    for index, row in nodes_df.iterrows():
        graph.add_node(row['Label'], name=row['Abb'])

    # Add edges with weights and attributes for the specific year
    for index, row in edges_df.iterrows():
        source = row['source']
        target = row['target']
        weight = row[migration_year]
        if weight != 0:
            female_ratio = row[migration_year.replace('total', 'female')] / weight
        else:
            female_ratio = 0
        graph.add_edge(source, target, weight=weight, female_ratio=female_ratio)

    # Clean up the graph (removing low weight edges and isolated nodes)
    mean_weight = np.mean([d['weight'] for _, _, d in graph.edges(data=True)])
    print(f'Mean weight: {mean_weight}')
    # remove_low_weight_edges(graph, mean_weight)
    # remove_isolated_nodes(graph)
    return graph

def visualize_graph(graph, nodes_df, title):
    node_sizes = [graph.degree(n, weight='weight')/5000 for n in graph.nodes()]
    edge_weights = [d['weight'] / 100000 for _, _, d in graph.edges(data=True)]

    # Position, size, and labels
    node_positions = {row['Label']: (row['lng'], row['lat']) for index, row in nodes_df.iterrows() if row['Label'] in graph.nodes}
    labels = get_high_degree_labels(graph, nodes_df, 100000)

    # Edge colors based on female ratio
    female_ratios = np.array([d['female_ratio'] for _, _, d in graph.edges(data=True)])
    norm = mcolors.Normalize(vmin=female_ratios.min(), vmax=female_ratios.max())
    cmap = plt.get_cmap('magma')
    edge_colors = [cmap(norm(d['female_ratio'])) for _, _, d in graph.edges(data=True)]

    # Visualization
    plt.figure(figsize=(15, 10))
    nx.draw_networkx_nodes(graph, node_positions, node_size=node_sizes, node_color='red', alpha=0.7)
    nx.draw_networkx_edges(graph, node_positions, width=edge_weights, alpha=0.5, edge_color=edge_colors)
    nx.draw_networkx_labels(graph, node_positions, labels=labels, font_size=12)
    plt.title(title)
    plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=plt.gca(), orientation='vertical', label='Female Ratio')
    plt.show()

def remove_low_weight_edges(graph, threshold):
    edges_to_remove = [(u, v) for u, v, d in graph.edges(data=True) if d['weight'] < threshold]
    graph.remove_edges_from(edges_to_remove)

def remove_isolated_nodes(graph):
    nodes_to_remove = [n for n in graph.nodes() if graph.degree(n) == 0]
    graph.remove_nodes_from(nodes_to_remove)

def get_high_degree_labels(graph, nodes_df, threshold=100000):
    return {n: data['Abb'] for n, data in nodes_df.set_index('Label').iterrows() 
            if n in graph.nodes() and (graph.degree(n, weight='weight') > threshold )}

# Main execution loop for each migration year
migration_years = [ 'migration_1995_total', 'migration_2000_total', 'migration_2005_total', 'migration_2010_total', 'migration_2015_total']
graphs =[]
for year in migration_years:
    # Create a separate graph for each year and store it in a dictionary
    graph = create_graph(nodes_df, edges_df, year)
    # Visualize the graph
    # visualize_graph(graph, nodes_df, year)
    graphs.append(graph)

Mean weight: 3124.0674207338798
Mean weight: 2759.10402565016
Mean weight: 3200.539989312433
Mean weight: 3692.077841111507
Mean weight: 2743.3322052012827


### Extracting BB networks using the disparity filter

In [2]:
from scipy import integrate


def disparity_filter(G, alpha=0.05):
    # Create an empty graph for the backbone
    B = nx.Graph()

    for node in G.nodes(data=True):
        node_id = node[0]  # Node identifier
        node_attr = node[1]  # Node attributes

        # Add the node to the backbone graph with its attributes
        B.add_node(node_id, **node_attr)

        # Calculate the total weight (strength) of the node
        s = sum([G[node_id][neighbor]['weight'] for neighbor in G.neighbors(node_id)])
        k = G.degree(node_id)

        for neighbor in G.neighbors(node_id):
            weight = G[node_id][neighbor]['weight']
            p_ij = weight / s

            # Calculate the disparity measure
            alpha_ij = 1 - (k - 1) * integrate.quad(lambda x: (1 - x)**(k - 2), 0, p_ij)[0]

            # Add edge to the backbone if significant, with its attributes
            if alpha_ij < alpha:
                edge_attr = G[node_id][neighbor]
                B.add_edge(node_id, neighbor, **edge_attr)

    return B

In [3]:
# Select alpha value
alpha = 0.001

# Extracting the backbone network for each graph in the list 'graphs'
backbone_graphs = [disparity_filter(graph, alpha) for graph in graphs]

In [4]:
# removing the isolated nodes  
for graph in backbone_graphs:
    remove_isolated_nodes(graph)

### Directed BB graphs without weights

In [5]:
# adding the directionality to the backbone network from the original graph
directed_backbone_graphs = []

for graph, backbone_graph in zip(graphs, backbone_graphs):
    directed_backbone_graph = nx.DiGraph()
    for node in backbone_graph.nodes(data=True):
        node_id = node[0]
        node_attr = node[1]
        directed_backbone_graph.add_node(node_id, **node_attr)

    for edge in backbone_graph.edges(data=True):
        source = edge[0]
        target = edge[1]
        directed_backbone_graph.add_edge(source, target)
    directed_backbone_graphs.append(directed_backbone_graph)

In [8]:
# visualizing the directed backbone network
for graph, year in zip(directed_backbone_graphs, migration_years):
    visualize_graph(graph, nodes_df, year)
    

TypeError: Bad graph type, use only non directed graph

In [6]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def visualize_backbone_graph(graph, nodes_df, title):
    # Calculate node sizes based on degree and weight
    node_sizes = [graph.degree(n, weight='weight')/5000 for n in graph.nodes()]
    edge_weights = [d['weight'] / 100000 for _, _, d in graph.edges(data=True)]

    # Position, size, and labels
    node_positions = {row['Label']: (row['lng'], row['lat']) for index, row in nodes_df.iterrows() if row['Label'] in graph.nodes}
    labels = get_high_degree_labels(graph, nodes_df, 200000)

    # Edge colors based on female ratio
    female_ratios = np.array([d['female_ratio'] for _, _, d in graph.edges(data=True)])
    norm = mcolors.Normalize(vmin=female_ratios.min(), vmax=female_ratios.max())
    cmap = plt.get_cmap('magma')
    edge_colors = [cmap(norm(d['female_ratio'])) for _, _, d in graph.edges(data=True)]

    # Visualization
    plt.figure(figsize=(15, 10))
    nx.draw_networkx_nodes(graph, node_positions, node_size=node_sizes, node_color='red', alpha=0.7)
    nx.draw_networkx_edges(graph, node_positions, width=edge_weights, alpha=0.5, edge_color=edge_colors)
    nx.draw_networkx_labels(graph, node_positions, labels=labels, font_size=12)
    plt.title(title)
    plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=plt.gca(), orientation='vertical', label='Female Ratio')
    plt.show()

# Visualize the first backbone network
visualize_backbone_graph(directed_backbone_graphs[0], nodes_df, "Backbone Network - Migration 1995")


KeyError: 'weight'